In [2]:
import json
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [3]:
client_id = "5a53b08bc80c4eee9c9e951aa6053549"
client_secret = "fe9f75fa5ad74bb28b764155b16f7cbd"
#playlist_id = "37i9dQZF1EM5Yxpzq4dCTr" #top 100
playlist_id = "1LfBD861oZjhOcKx2ugNTe" #master playlist 554 songs

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [5]:
# GET ALL TRACKS FROM PLAYLIST
results = sp.playlist_tracks(playlist_id)
tracks = results['items']
while results['next']:
    results = sp.next(results)
    tracks.extend(results['items'])

In [6]:
 display(results)

{'href': 'https://api.spotify.com/v1/playlists/1LfBD861oZjhOcKx2ugNTe/tracks?offset=700&limit=100&additional_types=track',
 'items': [{'added_at': '2023-03-31T09:14:24Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/22fln4iyazryy3lqq3nnqjufq'},
    'href': 'https://api.spotify.com/v1/users/22fln4iyazryy3lqq3nnqjufq',
    'id': '22fln4iyazryy3lqq3nnqjufq',
    'type': 'user',
    'uri': 'spotify:user:22fln4iyazryy3lqq3nnqjufq'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_group': 'album',
     'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/04gDigrS5kc9YWfZHwBETP'},
       'href': 'https://api.spotify.com/v1/artists/04gDigrS5kc9YWfZHwBETP',
       'id': '04gDigrS5kc9YWfZHwBETP',
       'name': 'Maroon 5',
       'type': 'artist',
       'uri': 'spotify:artist:04gDigrS5kc9YWfZHwBETP'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
   

In [14]:
len(tracks) # should be 554 for master playlist

725

In [16]:
# create a list of song ids
ids=[]


for track in tracks:
    track_id = track['track']['id']
    ids.append(track_id)
        
# check the songs audio feature
# CAN ONLY QUERY 100 SONGS AT A TIME
features = []
features.extend(sp.audio_features(ids[0:100]))
features.extend(sp.audio_features(ids[100:200]))
features.extend(sp.audio_features(ids[200:300]))
features.extend(sp.audio_features(ids[300:400]))
features.extend(sp.audio_features(ids[400:500]))
features.extend(sp.audio_features(ids[500:600]))
features.extend(sp.audio_features(ids[600:700]))
features.extend(sp.audio_features(ids[700:725]))

display(features)
# change dictionary to dataframe
features_df=pd.DataFrame.from_dict(features)

len(features_df)

[{'danceability': 0.66,
  'energy': 0.45,
  'key': 4,
  'loudness': -11.011,
  'mode': 1,
  'speechiness': 0.0451,
  'acousticness': 0.839,
  'instrumentalness': 0.714,
  'liveness': 0.374,
  'valence': 0.242,
  'tempo': 145.011,
  'type': 'audio_features',
  'id': '42NB7w0tDCag0mQpCtZe7Y',
  'uri': 'spotify:track:42NB7w0tDCag0mQpCtZe7Y',
  'track_href': 'https://api.spotify.com/v1/tracks/42NB7w0tDCag0mQpCtZe7Y',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/42NB7w0tDCag0mQpCtZe7Y',
  'duration_ms': 194547,
  'time_signature': 4},
 {'danceability': 0.542,
  'energy': 0.749,
  'key': 6,
  'loudness': -8.333,
  'mode': 0,
  'speechiness': 0.0277,
  'acousticness': 0.173,
  'instrumentalness': 0.942,
  'liveness': 0.315,
  'valence': 0.674,
  'tempo': 94.031,
  'type': 'audio_features',
  'id': '7Cg3F9ZsZ2TYUnlza49NYh',
  'uri': 'spotify:track:7Cg3F9ZsZ2TYUnlza49NYh',
  'track_href': 'https://api.spotify.com/v1/tracks/7Cg3F9ZsZ2TYUnlza49NYh',
  'analysis_url': 'https://api.

725

In [17]:
# extract only the numeric features
music_features = features_df[['danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'tempo']]

#display(music_features)

min_max_scaler = MinMaxScaler()

music_features[music_features.columns] = min_max_scaler.fit_transform(music_features[music_features.columns])

#display(music_features)
len(music_features)

C:\Users\sidv1\AppData\Local\Temp\ipykernel_19508\1442152539.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  music_features[music_features.columns] = min_max_scaler.fit_transform(music_features[music_features.columns])


725

In [18]:
#Write music_features and song_ids to csv and use 1:1 format

music_features.to_csv("music_features.csv", index=False)

id_df = pd.DataFrame(ids)
id_df.to_csv("song_ids.csv", header=False, index=False)
#music_features.to_excel("music_features.xlsx")

In [10]:
music_features2 = music_features[:1].copy(deep=True)

# display(music_features)
# display(music_features2)

#######################################
music_features2.loc[0, "danceability"] = 0.1
music_features2.loc[0, "energy"] = 0.3
music_features2.loc[0, "speechiness"] = 0.1
music_features2.loc[0, "acousticness"] = 0.1
music_features2.loc[0, "instrumentalness"] = 0.9
music_features2.loc[0, "tempo"] = 0.9
########################################

# display(music_features2)

cosine_sim = cosine_similarity(music_features, music_features2)
cosine_sim = cosine_sim.flatten()


# Pick the top 6 most similar songs
max6index = np.argsort(cosine_sim)[-6:][::-1]

rec_ids = []
for idx in max6index:
    rec_ids.append(sp.track(ids[idx]))
display(rec_ids)


[{'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/21mKp7DqtSNHhCAU2ugvUw'},
     'href': 'https://api.spotify.com/v1/artists/21mKp7DqtSNHhCAU2ugvUw',
     'id': '21mKp7DqtSNHhCAU2ugvUw',
     'name': 'ODESZA',
     'type': 'artist',
     'uri': 'spotify:artist:21mKp7DqtSNHhCAU2ugvUw'}],
   'available_markets': ['AD',
    'AE',
    'AL',
    'AR',
    'AT',
    'AU',
    'BA',
    'BE',
    'BG',
    'BH',
    'BO',
    'BR',
    'BY',
    'CA',
    'CH',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DE',
    'DK',
    'DO',
    'DZ',
    'EC',
    'EE',
    'EG',
    'ES',
    'FI',
    'FR',
    'GB',
    'GR',
    'GT',
    'HK',
    'HN',
    'HR',
    'HU',
    'ID',
    'IE',
    'IL',
    'IN',
    'IS',
    'IT',
    'JO',
    'KW',
    'KZ',
    'LB',
    'LI',
    'LT',
    'LU',
    'LV',
    'MA',
    'MC',
    'MD',
    'ME',
    'MK',
    'MT',
    'MX',
    'MY',
    'NI',
    'NL',
    'NO',
    'NZ'

In [11]:
embed = []
string = ''
for items in rec_ids:
    string = items['external_urls']['spotify']
    string = string[:25] + 'embed/' + string[25:]
    embed.append(string)
print(embed)


['https://open.spotify.com/embed/track/1zyMwU1NtRDkuQWthUbGsL', 'https://open.spotify.com/embed/track/2W2e6TBrwJt4ASh5jyYgvZ', 'https://open.spotify.com/embed/track/5IqjAH2ynF7320IVnqfjlT', 'https://open.spotify.com/embed/track/31NiyZrUd1r4icY7xkvnWv', 'https://open.spotify.com/embed/track/3iNj5fdyY62N1ycufQ3E2M', 'https://open.spotify.com/embed/track/3Le7mfkPuS7lNLpYY3Luzs']
